# Experiment 094: Ens-Model Kernel Exact Implementation

**Goal**: Implement the ens-model kernel exactly as shown in the public kernel.

**Key Features**:
1. ALL feature sources (spange, acs_pca, drfps, fragprints, smiles)
2. Correlation-based filtering with priority (spange > acs > drfps > frag > smiles)
3. CatBoost + XGBoost ensemble with weights: single (7:6), full (1:2)
4. Clipping to non-negative and renormalization if sum > 1
5. Engineered numeric features (T_x_RT, RT_log, T_inv, RT_scaled)

In [1]:
import numpy as np
import pandas as pd
import torch
from abc import ABC, abstractmethod
from functools import reduce
import warnings
warnings.filterwarnings('ignore')

torch.set_default_dtype(torch.double)

print("Imports complete")

Imports complete


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Base classes
class SmilesFeaturizer(ABC):
    def __init__(self):
        raise NotImplementedError

    def featurize(self, X):
        raise NotImplementedError

class BaseModel(ABC):
    def __init__(self):
        pass

    def train_model(self, X_train, y_train):
        raise NotImplementedError

    def predict(self, X):
        raise NotImplementedError

In [4]:
# Feature priority and correlation filtering (from ens-model kernel)
_SOLVENT_TABLE_CACHE = None

def feature_priority(name):
    """Assign priority score to feature name based on prefix."""
    if name.startswith("spange_"):
        return 5
    if name.startswith("acs_"):
        return 4
    if name.startswith("drfps_"):
        return 3
    if name.startswith("frag_"):
        return 2
    if name.startswith("smiles_"):
        return 1
    return 0

def filter_correlated_features(df, threshold=0.8):
    """Drop columns that are highly correlated with any other column."""
    numeric_df = df.select_dtypes(include=[np.number])
    
    if numeric_df.shape[1] == 0:
        return df, []
    
    # Drop constant columns first
    std = numeric_df.std(axis=0)
    constant_cols = std[std == 0].index.tolist()
    if constant_cols:
        numeric_df = numeric_df.drop(columns=constant_cols)
    
    # Correlation matrix
    corr = numeric_df.corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool)).fillna(0.0)
    
    cols = upper.columns.tolist()
    to_drop = set()
    
    # Find all pairs with corr > threshold
    high_corr_pairs = []
    for i, col_i in enumerate(cols):
        for j in range(i + 1, len(cols)):
            col_j = cols[j]
            cval = upper.iloc[i, j]
            if cval > threshold:
                high_corr_pairs.append((col_i, col_j, cval))
    
    # For each pair, decide which column to drop
    for col_i, col_j, cval in high_corr_pairs:
        if col_i in to_drop or col_j in to_drop:
            continue
        
        p_i = feature_priority(col_i)
        p_j = feature_priority(col_j)
        
        if p_i > p_j:
            drop = col_j
        elif p_j > p_i:
            drop = col_i
        else:
            idx_i = df.columns.get_loc(col_i)
            idx_j = df.columns.get_loc(col_j)
            drop = col_i if idx_i > idx_j else col_j
        
        to_drop.add(drop)
    
    all_to_drop = list(set(constant_cols).union(to_drop))
    df_filtered = df.drop(columns=all_to_drop, errors="ignore")
    
    return df_filtered, all_to_drop

print("Feature filtering functions defined")

Feature filtering functions defined


In [5]:
# Numeric feature engineering
def add_numeric_features(X_numeric):
    """Add engineered numeric features."""
    X_num = X_numeric.copy()
    cols = set(X_num.columns)
    
    if {"Temperature", "Residence Time"} <= cols:
        # Convert Temperature to Kelvin
        X_num["Temperature"] = X_num["Temperature"] + 273.15
        
        T = X_num["Temperature"]
        rt = X_num["Residence Time"]
        
        # Interaction term
        X_num["T_x_RT"] = T * rt
        
        # Log transformation
        X_num["RT_log"] = np.log(rt + 1e-6)
        
        # Inverse temperature
        X_num["T_inv"] = 1 / T
        
        # Scaled residence time
        X_num["RT_scaled"] = rt / rt.mean()
    
    return X_num

print("Numeric feature engineering defined")

Numeric feature engineering defined


In [6]:
# Build solvent feature table - simplified version
def build_solvent_feature_table(threshold=0.90):
    """Build combined solvent feature table from multiple sources."""
    global _SOLVENT_TABLE_CACHE
    
    if _SOLVENT_TABLE_CACHE is not None:
        return _SOLVENT_TABLE_CACHE
    
    print(">>> Building solvent feature table...")
    
    # Load each feature source
    spange = load_features("spange_descriptors")
    acs_pca = load_features("acs_pca_descriptors")
    drfps = load_features("drfps_catechol")
    
    # Rename columns with prefixes
    spange.columns = [f"spange_{c}" for c in spange.columns]
    acs_pca.columns = [f"acs_{c}" for c in acs_pca.columns]
    drfps.columns = [f"drfps_{c}" for c in drfps.columns]
    
    print(f"Spange shape: {spange.shape}")
    print(f"ACS PCA shape: {acs_pca.shape}")
    print(f"DRFPS shape: {drfps.shape}")
    
    # Merge all sources
    merged = pd.concat([spange, acs_pca, drfps], axis=1)
    
    # Keep only numeric columns
    merged = merged.select_dtypes(include=[np.number])
    
    print(f"Merged shape before filtering: {merged.shape}")
    
    # Filter DRFP to non-zero variance columns
    variance = merged.var()
    nonzero_var_cols = variance[variance > 0].index.tolist()
    merged = merged[nonzero_var_cols]
    
    print(f"Merged shape after variance filtering: {merged.shape}")
    
    # Apply correlation filtering
    merged_filtered, dropped = filter_correlated_features(merged, threshold=threshold)
    
    print(f"Merged shape after correlation filtering: {merged_filtered.shape}")
    print(f"Dropped {len(dropped)} columns")
    
    _SOLVENT_TABLE_CACHE = merged_filtered
    return _SOLVENT_TABLE_CACHE

print("Solvent feature table builder defined")

Solvent feature table builder defined


In [7]:
# Featurizers
class PrecomputedFeaturizer(SmilesFeaturizer):
    """Featurizer for single-solvent data."""
    
    def __init__(self):
        self.solvent_table = build_solvent_feature_table(threshold=0.90)
    
    def featurize(self, X):
        X_numeric = X[["Residence Time", "Temperature"]].copy()
        X_numeric = add_numeric_features(X_numeric)
        
        solvent_names = X["SOLVENT NAME"]
        X_solvent = self.solvent_table.loc[solvent_names].reset_index(drop=True)
        
        X_combined = pd.concat([X_numeric.reset_index(drop=True), X_solvent], axis=1)
        
        return torch.tensor(X_combined.values, dtype=torch.double)

class PrecomputedFeaturizerMixed(SmilesFeaturizer):
    """Featurizer for mixed-solvent data."""
    
    def __init__(self):
        self.solvent_table = build_solvent_feature_table(threshold=0.90)
    
    def featurize(self, X):
        X_numeric = X[["Residence Time", "Temperature"]].copy()
        X_numeric = add_numeric_features(X_numeric)
        
        pct_b = X["SolventB%"].values.reshape(-1, 1)
        
        solvent_a_names = X["SOLVENT A NAME"]
        solvent_b_names = X["SOLVENT B NAME"]
        
        X_a = self.solvent_table.loc[solvent_a_names].values
        X_b = self.solvent_table.loc[solvent_b_names].values
        
        # Weighted average of solvent features
        X_solvent = (1 - pct_b) * X_a + pct_b * X_b
        X_solvent_df = pd.DataFrame(X_solvent, columns=self.solvent_table.columns)
        
        X_combined = pd.concat([X_numeric.reset_index(drop=True), X_solvent_df], axis=1)
        
        return torch.tensor(X_combined.values, dtype=torch.double)

print("Featurizers defined")

Featurizers defined


In [8]:
# CatBoost Model
from catboost import CatBoostRegressor

class CatBoostModel(BaseModel):
    """CatBoost model with per-target regressors."""
    
    def __init__(self, data='single', random_state=42, verbose=False):
        self.data_mode = data
        self.verbose = verbose
        self.random_state = random_state
        
        if data == 'single':
            self.smiles_featurizer = PrecomputedFeaturizer()
            self.cat_params = dict(
                random_state=random_state,
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3.0,
                verbose=False,
            )
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed()
            self.cat_params = dict(
                random_state=random_state,
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3.0,
                verbose=False,
            )
        
        self.models = None
        self.n_targets = None
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        X_tensor = self.smiles_featurizer.featurize(train_X)
        X_np = X_tensor.detach().cpu().numpy()
        Y_np = train_Y.values
        self.n_targets = Y_np.shape[1]
        
        self.models = []
        for t in range(self.n_targets):
            m = CatBoostRegressor(**self.cat_params)
            m.fit(X_np, Y_np[:, t])
            self.models.append(m)
    
    def predict(self, X):
        if self.models is None:
            raise RuntimeError("CatBoostModel not trained")
        
        X_tensor = self.smiles_featurizer.featurize(X)
        X_np = X_tensor.detach().cpu().numpy()
        
        preds_list = [m.predict(X_np) for m in self.models]
        out = np.column_stack(preds_list)
        
        # Clip to non-negative
        out = np.clip(out, a_min=0.0, a_max=None)
        
        # Renormalize if sum > 1
        if out.shape[1] > 1:
            totals = out.sum(axis=1, keepdims=True)
            divisor = np.maximum(totals, 1.0)
            out = out / divisor
        
        return torch.tensor(out, dtype=torch.double)

print("CatBoostModel defined")

CatBoostModel defined


In [9]:
# XGBoost Model
import xgboost as xgb

class XGBModel(BaseModel):
    """XGBoost model with per-target regressors."""
    
    def __init__(self, data='single', random_state=42, verbose=False):
        self.data_mode = data
        self.verbose = verbose
        self.random_state = random_state
        
        if data == 'single':
            self.smiles_featurizer = PrecomputedFeaturizer()
            self.xgb_params = dict(
                random_state=random_state,
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=1.0,
                reg_alpha=0.1,
                verbosity=0,
            )
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed()
            self.xgb_params = dict(
                random_state=random_state,
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_lambda=1.0,
                reg_alpha=0.1,
                verbosity=0,
            )
        
        self.models = None
        self.n_targets = None
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        X_tensor = self.smiles_featurizer.featurize(train_X)
        X_np = X_tensor.detach().cpu().numpy()
        Y_np = train_Y.values
        self.n_targets = Y_np.shape[1]
        
        self.models = []
        for t in range(self.n_targets):
            m = xgb.XGBRegressor(**self.xgb_params)
            m.fit(X_np, Y_np[:, t])
            self.models.append(m)
    
    def predict(self, X):
        if self.models is None:
            raise RuntimeError("XGBModel not trained")
        
        X_tensor = self.smiles_featurizer.featurize(X)
        X_np = X_tensor.detach().cpu().numpy()
        
        preds_list = [m.predict(X_np) for m in self.models]
        out = np.column_stack(preds_list)
        
        # Clip to non-negative
        out = np.clip(out, a_min=0.0, a_max=None)
        
        # Renormalize if sum > 1
        if out.shape[1] > 1:
            totals = out.sum(axis=1, keepdims=True)
            divisor = np.maximum(totals, 1.0)
            out = out / divisor
        
        return torch.tensor(out, dtype=torch.double)

print("XGBModel defined")

XGBModel defined


In [10]:
# Ensemble Model
class EnsembleModel(BaseModel):
    """Weighted ensemble of CatBoost and XGBoost."""
    
    def __init__(self, data='single', verbose=False):
        self.data_mode = data
        self.verbose = verbose
        
        # Optimized weights per dataset (from ens-model kernel)
        if data == 'single':
            cat_weight = 7.0
            xgb_weight = 6.0
        else:
            cat_weight = 1.0
            xgb_weight = 2.0
        
        # Normalize weights
        w_sum = cat_weight + xgb_weight
        self.cat_weight = cat_weight / w_sum
        self.xgb_weight = xgb_weight / w_sum
        
        # Initialize base models
        self.cat_model = CatBoostModel(data=data)
        self.xgb_model = XGBModel(data=data)
    
    def train_model(self, train_X, train_Y, device=None, verbose=False):
        self.cat_model.train_model(train_X, train_Y)
        self.xgb_model.train_model(train_X, train_Y)
    
    def predict(self, X):
        cat_pred = self.cat_model.predict(X)
        xgb_pred = self.xgb_model.predict(X)
        
        out = self.cat_weight * cat_pred + self.xgb_weight * xgb_pred
        
        return out

print("EnsembleModel defined")

EnsembleModel defined


In [11]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape}, {Y_single.shape}")

model = EnsembleModel(data='single')
model.train_model(X_single, Y_single)
preds = model.predict(X_single[:5])
print(f"Test predictions shape: {preds.shape}")
print(f"Sample predictions:\n{preds[:3]}")

Single solvent data: (656, 3), (656, 3)
>>> Building solvent feature table...
Spange shape: (26, 13)
ACS PCA shape: (24, 5)
DRFPS shape: (24, 2048)
Merged shape before filtering: (26, 2066)
Merged shape after variance filtering: (26, 140)
Merged shape after correlation filtering: (26, 56)
Dropped 84 columns


Test predictions shape: torch.Size([5, 3])
Sample predictions:
tensor([[0.0000, 0.0000, 0.9116],
        [0.0038, 0.0000, 0.9076],
        [0.0135, 0.0077, 0.8917]])


In [12]:
# Run CV
import tqdm

def compute_cv_score(verbose=True):
    """Compute CV score with ens-model approach."""
    
    # Single solvent CV
    X_single, Y_single = load_data("single_solvent")
    split_generator = generate_leave_one_out_splits(X_single, Y_single)
    
    single_mse_list = []
    for fold_idx, split in enumerate(split_generator):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = EnsembleModel(data='single')
        model.train_model(train_X, train_Y)
        
        predictions = model.predict(test_X)
        predictions_np = predictions.detach().cpu().numpy()
        
        mse = np.mean((predictions_np - test_Y.values) ** 2)
        single_mse_list.append(mse)
        if verbose:
            print(f"Single Fold {fold_idx}: MSE = {mse:.6f}")
    
    single_cv = np.mean(single_mse_list)
    if verbose:
        print(f"\nSingle Solvent CV MSE: {single_cv:.6f}")
    
    # Full data CV
    X_full, Y_full = load_data("full")
    split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
    
    full_mse_list = []
    for fold_idx, split in enumerate(split_generator):
        (train_X, train_Y), (test_X, test_Y) = split
        
        model = EnsembleModel(data='full')
        model.train_model(train_X, train_Y)
        
        predictions = model.predict(test_X)
        predictions_np = predictions.detach().cpu().numpy()
        
        mse = np.mean((predictions_np - test_Y.values) ** 2)
        full_mse_list.append(mse)
        if verbose:
            print(f"Full Fold {fold_idx}: MSE = {mse:.6f}")
    
    full_cv = np.mean(full_mse_list)
    if verbose:
        print(f"\nFull Data CV MSE: {full_cv:.6f}")
    
    combined_cv = (single_cv + full_cv) / 2
    if verbose:
        print(f"\n=== Combined CV MSE: {combined_cv:.6f} ===")
    
    return single_cv, full_cv, combined_cv

print("Running CV...")
single_cv, full_cv, combined_cv = compute_cv_score()

Running CV...


Single Fold 0: MSE = 0.031256


Single Fold 1: MSE = 0.024186


Single Fold 2: MSE = 0.001820


Single Fold 3: MSE = 0.019318


...

Full Fold 8: MSE = 0.002045


Full Fold 9: MSE = 0.005347


Full Fold 10: MSE = 0.003088


Full Fold 11: MSE = 0.004331


Full Fold 12: MSE = 0.002400

Full Data CV MSE: 0.008695

=== Combined CV MSE: 0.009239 ===


In [13]:
# Save results
import json

results = {
    'cv_score': float(combined_cv),
    'single_cv': float(single_cv),
    'full_cv': float(full_cv),
    'model': 'EnsembleModel (CatBoost + XGBoost with correlation filtering)',
    'baseline_cv': 0.008298,
    'improvement': f"{(0.008298 - combined_cv) / 0.008298 * 100:.2f}%"
}

with open('/home/code/experiments/094_ens_model_exact/metrics.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved")
print(f"Combined CV: {combined_cv:.6f}")
print(f"Baseline CV: 0.008298")
print(f"Improvement: {(0.008298 - combined_cv) / 0.008298 * 100:.2f}%")

Results saved
Combined CV: 0.009239
Baseline CV: 0.008298
Improvement: -11.34%


## Generate Submission

The following cells follow the official template structure.

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = EnsembleModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = EnsembleModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

# Also save to standard location
import shutil
import os
os.makedirs('/home/submission', exist_ok=True)
shutil.copy("submission.csv", "/home/submission/submission.csv")
print("Submission saved!")
print(f"Shape: {submission.shape}")
print(submission.head())

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################